In [139]:
#Eduardo Barroso Ribeiro Fernandes
#N_USP: 12681733
import numpy as np
import imageio
import matplotlib.pyplot as plt
import statistics as st

In [140]:

def filtering_1d(array_img, array_w, w_size):
    a_size = int(array_img.size)
    array = np.asarray(array_img).flatten()
    arr = np.zeros(a_size)
    for i in range(a_size):
        for j in range(w_size):
            #When reach de end of array
            if i+j >= a_size:
                #Back to the start
                arr[i] += array[j-1]*array_w[j]
            else:
                arr[i] += array[i+j]*array_w[j]

    #Correcting array:
    
    #The last to start
    arr = np.insert(arr, 0, arr[a_size-1], axis = 0)

    #Slicing until the penultimate
    arr = arr[:a_size]

    #Reshaping to a bidimesional array
    arr = np.reshape(arr,(int(np.sqrt(a_size)),int(np.sqrt(a_size))))

    return arr


In [141]:
def arr_dot(array1, array2):
  return np.sum(np.multiply(array1,array2))

In [152]:
def binarize(img, img_size, threshold):
  for i in range (img_size):
    for j in range (img_size):
      if img[i][j] > threshold:
        img[i][j] = 1
      else:
        img[i][j] = 0
  return img

In [153]:
def limiarization(img,img_size, threshold):

  #Separating the two regions of different intesities 
  avg1 = np.average(img[np.where(img > threshold)])
  avg2 = np.average(img[np.where(img <= threshold)])
  t = (avg1+avg2)/2

  if np.abs(t - threshold) > 0.5:

    #Recursive call to reach the theshold pretended
    return limiarization(img, img_size, t)

  else:

    return binarize(img, img_size, t)

In [144]:
def filtering_2d(img, img_size, weights, size_filter, threshold):
  #the value of Pad (to the border) is the whole number of size_filter/2
  pad = size_filter//2

  #aplying the borders as a symmetric matrix
  img = np.pad(img, pad_width=pad, mode='symmetric')

  arr = np.zeros((img_size,img_size))
  for i in range(pad,img_size+pad):
    for j in range(pad,img_size+pad):

      #Summing the term-by-term multiplications of the arrays
      arr[i-pad][j-pad] = np.sum(np.multiply(weights,img[i-pad:i+pad+1,j-pad:j+pad+1]))
      
  return limiarization(arr, img_size, threshold)

In [145]:
def median_filter(img, img_size, size_filter):
  #the value of Pad (to the border) is the whole number of size_filter/2
  pad = size_filter//2

  #aplying the borders
  img = np.pad(img, pad_width=pad, mode='constant', constant_values = 0)

  arr = np.zeros((img_size,img_size))
  for i in range(pad, img_size+pad):
    for j in range(pad, img_size+pad):
        #slicing array and calculating median value
        arr[i-pad][j-pad] = st.median(np.asarray(img[i-pad:i+pad+1,j-pad:j+pad+1]).flatten())

  return arr

In [146]:
#Normalization Values to int[0,255]
def normalization(img):
  min = np.min(img)
  max = np.max(img)
  img_new = np.asarray(((img - min)/(max-min))*255)
  return img_new

In [154]:
#Comparison with original image usin the Root mean square error
def error(img, output):
  MSE = np.square(np.subtract(img,output)).mean() 
  RMSE = np.sqrt(np.square(np.subtract(img,output)).mean())
  print(f'{RMSE:.4f}')

In [148]:
#Choosing method
def method_img(img, img_size, method):
  
    if method == 1:

        threshold = int(input())

        return limiarization(img,img_size,threshold)

    elif method == 2:

        n = int(input())
        weights = np.asarray(list(map(int,input().split())))

        return filtering_1d(np.asarray(img).flatten(), weights, n)

    elif method == 3:

        n = int(input())
        weights = np.zeros((n,n))

        for i in range(n):

          weights[i] = np.asarray(list(map(int,input().split())))

        threshold = int(input())

        return filtering_2d(img, img_size,weights,n, threshold)

    else:
      
        n = int(input())

        return median_filter(img, img_size, n)

In [149]:
#Input values
filename = str(input()).rstrip()
method = int(input())
img = imageio.imread(filename)
img_size = int(np.sqrt(np.size(img)))

lena_saltpepper.png
4


In [155]:
#Output image
output = method_img(img, img_size,method)
output = normalization(output)
error(img, output)

5
45.6618
